In [1]:
import pandas as pd
import numpy as np
import random
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import scale
from functools import partial
import pylab
import numpy as np
import numpy.linalg as npla

data = pd.read_csv("letter-recognition.csv")
print("Dimensions: ", data.shape, "\n")
print(data.info())
data.head()

Dimensions:  (20000, 17) 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 17 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   letter  20000 non-null  object
 1   xbox    20000 non-null  int64 
 2   ybox    20000 non-null  int64 
 3   width   20000 non-null  int64 
 4   height  20000 non-null  int64 
 5   onpix   20000 non-null  int64 
 6   xbar    20000 non-null  int64 
 7   ybar    20000 non-null  int64 
 8   x2bar   20000 non-null  int64 
 9   y2bar   20000 non-null  int64 
 10  xybar   20000 non-null  int64 
 11  x2br    20000 non-null  int64 
 12  xy2br   20000 non-null  int64 
 13  xege    20000 non-null  int64 
 14  xegvy   20000 non-null  int64 
 15  yege    20000 non-null  int64 
 16  yegvx   20000 non-null  int64 
dtypes: int64(16), object(1)
memory usage: 2.6+ MB
None


,letter,xbox,ybox,width,height,onpix,xbar,ybar,x2bar,y2bar,xybar,x2br,xy2br,xege,xegvy,yege,yegvx
0,T,2,8,3,5,1,8,13,0,6,6,10,8,0,8,0,8
1,I,5,12,3,7,2,10,5,5,4,13,3,9,2,8,4,10
2,D,4,11,6,8,6,10,6,2,6,10,3,7,3,7,3,9
3,N,7,11,6,6,3,5,9,4,6,4,4,10,6,10,2,8
4,G,2,1,3,1,1,8,6,6,6,6,5,9,1,7,5,10


In [2]:
dataMean = data.groupby('letter').mean()
dataMean.head()

,xbox,ybox,width,height,onpix,xbar,ybar,x2bar,y2bar,xybar,x2br,xy2br,xege,xegvy,yege,yegvx
letter,,,,,,,,,,,,,,,,
A,3.337136,6.975919,5.128010,5.178707,2.991128,8.851711,3.631179,2.755387,2.043093,7.802281,2.338403,8.465146,2.771863,6.321926,2.875792,7.468948
B,3.985640,6.962141,5.088773,5.169713,4.596606,7.671018,7.062663,5.366841,5.571802,7.954308,5.506527,6.652742,3.117493,7.919060,6.612272,9.100522
C,4.031250,7.063859,4.701087,5.296196,2.775815,5.437500,7.627717,5.927989,7.177989,8.773098,7.494565,11.947011,1.991848,8.876359,4.080163,8.555707
D,4.023602,7.244720,5.170186,5.288199,4.026087,7.539130,6.806211,5.921739,6.508075,8.166460,5.111801,5.750311,3.365217,7.813665,3.971429,7.628571
E,3.727865,6.944010,4.756510,5.201823,3.679688,5.966146,7.352865,4.223958,7.585938,8.507812,6.242188,10.341146,2.127604,8.298177,6.022135,8.506510


In [3]:

round(data.drop('letter', axis=1).mean(), 2)


X = data.drop("letter", axis = 1)
y = data['letter']

scaleX = scale(X)

trainX, testX, trainY, testY = train_test_split(scaleX, y, test_size = 0.3, random_state = 101)


In [4]:
def markovSampling(trainX, YTrain):
    Init_sampl = np.concatenate((np.vstack(trainX), np.vstack(YTrain.to_numpy())), axis = 1)
    trainingSampleSet = random.sample(list(Init_sampl), 2000)
    M = len(trainingSampleSet)
    print("M : " , M)

    Mminus = 0
    Mplus = 0
    t=1

    i = np.random.choice(len(trainingSampleSet), 1, replace=False)  
    D = np.array(trainingSampleSet)
    currentSample = D[i][0]

    print("currentSample : ", currentSample)
    
    
    if currentSample[16]=='A':
        Mplus+=1
    else:
        Mminus+=1
    
    
    sample = []
    pone = 0
    ptwo = 0
    
    print(currentSample)
    k = 5
    q = 1.2
    
    while( Mminus + Mplus < M ):
        ZStar = D[np.random.choice(len(trainingSampleSet), 1, replace=False)][0] #it is random sample that is obtained from St
        Probability = 1
        yt = currentSample[16]
        currentSample = ZStar

        
        if Probability>=1 and currentSample[16]==yt:  # and  condition is missing..
            sample.append(ZStar) #Pdash = e−y∗f0 /e−ytf0
        else:
            sample.append(ZStar) #P
        
        if len(sample) == k:
            ptwo = q*Probability
            sample.append(ZStar) #Pdd
            
        

        if yt == 'A' and Mplus<(M/2):
            Mplus += 1
        else:
            if Mminus<(M/2):
                Mminus += 1

        if pone> 1 or Probability>1 or ptwo > 1:
            sample.append(ZStar)      
        
    return sample

In [5]:
Nsample = np.array(markovSampling(trainX, trainY))
print(Nsample.shape)
trainX = Nsample[:, 0:16]
trainY = Nsample[:, 16]

print(trainX.shape)
print(trainY.shape)

M :  2000
currentSample :  ['0.5103849674390448' '0.8971174160969361' '0.4359096603952119'
 '1.1619469387139192' '1.595208656581608' '-0.4430438069275676'
 '1.0749385844441806' '0.507944667966729' '-0.4950723031444648'
 '-0.11334534632124997' '-0.5526406630296021' '0.5147635260640492'
 '-0.019764349464260575' '1.0740075215176041' '1.6783155292803702'
 '1.9777058426628995' 'F']
['0.5103849674390448' '0.8971174160969361' '0.4359096603952119'
 '1.1619469387139192' '1.595208656581608' '-0.4430438069275676'
 '1.0749385844441806' '0.507944667966729' '-0.4950723031444648'
 '-0.11334534632124997' '-0.5526406630296021' '0.5147635260640492'
 '-0.019764349464260575' '1.0740075215176041' '1.6783155292803702'
 '1.9777058426628995' 'F']
(50113, 17)
(50113, 16)
(50113,)


In [6]:

YTrain = []
for i in trainY:
    YTrain.append(ord(i))

YTest = []
for i in testY:
    YTest.append(ord(i))


In [7]:
print(testX.shape)

(6000, 16)


In [8]:
def linearKernel():

    model_linear = SVC(kernel='linear')
    model_linear.fit(trainX, YTrain)
    YPred = model_linear.predict(testX)

 
    print("MCC:",  metrics.matthews_corrcoef(y_true = YTest, y_pred = YPred), "\n")
    print("accuracy:", metrics.accuracy_score(y_true = YTest, y_pred = YPred), "\n")
    print(metrics.confusion_matrix(y_true = YTest, y_pred = YPred))

def rbfkernel():

    non_linear_model = SVC(kernel='rbf')
    non_linear_model.fit(trainX, YTrain)
    YPred = non_linear_model.predict(testX)


    print("MCC:",  metrics.matthews_corrcoef(y_true=YTest, y_pred=YPred), "\n")
    print("accuracy:", metrics.accuracy_score(y_true=YTest, y_pred=YPred), "\n")
    print(metrics.confusion_matrix(y_true=YTest, y_pred=YPred))

def polykernel():
    non_linear_model = SVC(kernel='poly')
    non_linear_model.fit(trainX, YTrain)
    YPred = non_linear_model.predict(testX)


    print("MCC:", metrics.matthews_corrcoef(y_true=YTest, y_pred = YPred), "\n")
    print("accuracy:", metrics.accuracy_score(y_true= YTest, y_pred = YPred), "\n")
    print(metrics.confusion_matrix(y_true=YTest, y_pred= YPred))

In [9]:
print("Kernel = linear")
linearKernel()
print("")

print("Kernel = rbf")
rbfkernel()
print("")

print("Kernel = poly")
polykernel()
print("")

Kernel = linear
MCC: 0.7813795517331247 

accuracy: 0.7896666666666666 

[[186   0   0   0   0   0   1   3   0   0   1   2   1   1   0   0   3   1
    1   3   2   0   0   0   4   0]
 [  0 163   0   3   3   0   4   6   2   0   1   1   4   2   0   1   3   8
    8   0   0   7   0   0   2   0]
 [  0   0 186   0   9   1  11   3   0   0   6   6   0   0   5   0   0   0
    0   1   2   0   0   0   0   0]
 [  2   8   0 212   0   1   0   4   0   5   0   0   1   4   3   0   0   3
    0   3   0   0   0   1   0   1]
 [  0   2  11   0 196   0  11   1   0   0   1   7   0   0   0   0   0   0
    0   0   0   1   0   1   0   3]
 [  4   3   2   2   4 178   1   0   2   2   0   0   0   2   0   6   1   0
    7   6   0   1   1   1   2   0]
 [  1   3  20   3   5   1 161   5   1   0   1   2   4   0   2   1   5   0
    2   0   1   1   0   0   0   0]
 [  3   4   2  14   0   4   4 133   0   3   3   1   3   0  18   0   5   7
    0   2   5   4   0   2   0   0]
 [  1   0   0   1   0   4   0   0 181  16   1   1   0  

In [10]:

folds = KFold(n_splits = 5, shuffle = True, random_state = 101)
hyper_params = [ {'gamma': [1e-2, 1e-3, 1e-4],'C': [1, 10, 100, 1000]}]



model = SVC(kernel="rbf")

model_cv = GridSearchCV(estimator = model,param_grid = hyper_params,scoring= 'accuracy',cv = folds,verbose = 1,return_train_score=True)      


model_cv.fit(trainX, trainY)                  


Fitting 5 folds for each of 12 candidates, totalling 60 fits


KeyboardInterrupt: 

In [ ]:

Result['param_C'] = Result['param_C'].astype('int')


plt.figure(figsize=(24,6))


plt.subplot(131)
gamma_01 = Result[Result['param_gamma']==0.01]

plt.plot(gamma_01["param_C"], gamma_01["mean_test_score"])
plt.plot(gamma_01["param_C"], gamma_01["mean_train_score"])
plt.xlabel('C')
plt.ylabel('Accuracy')
plt.title("Gamma=0.01")
plt.ylim([0.60, 1])
plt.legend(['test accuracy', 'train accuracy'], loc='upper left')
plt.xscale('log')


plt.subplot(132)
gamma_001 = Result[Result['param_gamma']==0.001]

plt.plot(gamma_001["param_C"], gamma_001["mean_test_score"])
plt.plot(gamma_001["param_C"], gamma_001["mean_train_score"])
plt.xlabel('C')
plt.ylabel('Accuracy')
plt.title("Gamma=0.001")
plt.ylim([0.60, 1])
plt.legend(['test accuracy', 'train accuracy'], loc='upper left')
plt.xscale('log')



plt.subplot(133)
gamma_0001 = Result[Result['param_gamma']==0.0001]

plt.plot(gamma_0001["param_C"], gamma_0001["mean_test_score"])
plt.plot(gamma_0001["param_C"], gamma_0001["mean_train_score"])
plt.xlabel('C')
plt.ylabel('Accuracy')
plt.title("Gamma=0.0001")
plt.ylim([0.60, 1])
plt.legend(['test accuracy', 'train accuracy'], loc='upper left')
plt.xscale('log')


In [ ]:
best_score = model_cv.best_score_
best_hyperparams = model_cv.best_params_

print("The best test score = {0} corresponding to hyperparameters {1}".format(best_score, best_hyperparams))


model = SVC(C=1000, gamma=0.01, kernel="rbf")

model.fit(trainX, trainY)
YPred = model.predict(testX)


print("accuracy", metrics.accuracy_score(testY, YPred), "\n")
print(metrics.confusion_matrix(testY, YPred), "\n")
